In [2]:
#Problem 6-1
class Graph(object):
    class Vertex(object):
        __slots__='_elem'
        def __init__(self,x):
            self._elem = x
        #加上@property，方法变成属性，按照属性的方式调用，否则按照方法调用加()
        def elem(self):
            return self._elem
        def __hash__(self):
            #hash值，唯一确定顶点
            return hash(id(self))
        def __repr__(self):
            return str(self._elem)
    
    class Edge(object):
        __slots__='_src','_des','_elem'
        
        def __init__(self, u, v, x):
            self._src = u
            self._des = v
            self._elem = x
        def endpoints(self):
            #取两个端点
            return (self._src,self._des)
        def opposite(self,v):
            #已知一个端点和所在边，返回另一个端点
            if not isinstance(v,Graph.Vertex):
                raise TypeError('v must be a Vertex')
            return self._des if v is self._src else self._src
        def elem(self):
            return self._elem
        def __hash__(self):
            return hash((self._src,self._des))
        def __repr__(self):
            return f'({self._src},{self._des})'#,{self._elem})'
    
    def __init__(self,directed=False):
        self._outgoing={}#出边集合
        self._incoming={} if directed else self._outgoing#入边集合
    def _validate_vertex(self,v):
        #检验是不是顶点
        if not isinstance(v,self.Vertex):
            raise TypeError('Vertex expected')
        #检验顶点在不在图中，检验字典中的键，不能检验值，值是可变的
        if v not in self._outgoing:
            raise ValueError('Vertex does not belong to this graph.')
    def is_directed(self):
        return self._incoming is not self._outgoing
    @property
    def n_vertices(self):
        return len(self._outgoing)
    def vertices(self):
        return self._outgoing.keys()
    @property
    def n_edges(self):
        total=sum(len(self._outgoing[v]) for v in self._outgoing)
        return total if self.is_directed() else total // 2#无向图除2
    def edges(self):
        #返回所有的边
        result=set()
        for secondary_map in self._outgoing.values():#取键用keys取值用values
            result.update(secondary_map.values())
        return result
    def get_edge(self,u,v):
        #返回u-v的边
        self._validate_vertex(u)
        self._validate_vertex(v)
        return self._outgoing[u].get(v)
    def degree(self,v,outgoing=True):
        self._validate_vertex(v)
        adj = self._outgoing if outgoing else self._incoming
        return len(adj[v])
    def incident_edges(self, v, outgoing=True): 
        #返回所有和v邻接的边
        self._validate_vertex(v)
        adj = self._outgoing if outgoing else self. _incoming
        for edge in adj[v].values():
            yield edge
    def insert_vertex(self, x=None): 
        #插入顶点
        v = self.Vertex(x)
        self._outgoing[v] = {}
        if self.is_directed(): 
            self._incoming[v] = {} 
        return v
    def insert_edge(self, u, v, x=None):
        if self.get_edge(u, v) is not None:
            raise ValueError('u and v are already adjacent')
        e = self.Edge(u, v, x)
        self._outgoing[u][v] = e
        self._incoming[v][u] = e
    def __repr__(self):
        if self.is_directed():
            return f'{self._outgoing}{self._incoming}'
        else:
            return f'{self._outgoing}'
        


In [3]:
#Problem
def DFS(g, u, discovered):
    for e in g.incident_edges(u): 
        v = e.opposite(u)
        if v not in discovered: 
            discovered[v] = e
            DFS(g, v, discovered)

def construct_path(u, v, discovered):
    path = []
    if v in discovered:
        path.append(v)
        walk=v
        while walk is not u:
            e=discovered[walk]
            parent=e.opposite(walk)
            path.append(parent)
            walk=parent
        path.reverse()
    return path

In [11]:
if __name__ =='__main__':
    G=Graph()

    a=G.insert_vertex('a')
    b=G.insert_vertex('b')
    c=G.insert_vertex('c')
    d=G.insert_vertex('d')
    e=G.insert_vertex('e')
    f=G.insert_vertex('f')
    
    G.insert_edge(a,b)
    G.insert_edge(a,c)
    G.insert_edge(a,d)
    G.insert_edge(a,f)
    G.insert_edge(b,c)
    G.insert_edge(c,d)
    G.insert_edge(c,e)
    G.insert_edge(b,e)
    G.insert_edge(b,f)
    
    print(G)
    print(G.n_vertices)
    print(G.n_edges)
    print(list(G.vertices()))
    print(list(G.edges()))
    
    bfs_tree={a:None}
    DFS(G,a,bfs_tree)
    print(bfs_tree)
    
    path=construct_path(a,e,bfs_tree)
    print(f'a path from a to e is {path}')

{a: {b: (a,b), c: (a,c), d: (a,d), f: (a,f)}, b: {a: (a,b), c: (b,c), e: (b,e), f: (b,f)}, c: {a: (a,c), b: (b,c), d: (c,d), e: (c,e)}, d: {a: (a,d), c: (c,d)}, e: {c: (c,e), b: (b,e)}, f: {a: (a,f), b: (b,f)}}
6
9
[a, b, c, d, e, f]
[(b,c), (a,d), (a,c), (b,e), (a,b), (a,f), (b,f), (c,d), (c,e)]
{a: None, b: (a,b), c: (b,c), d: (c,d), e: (c,e), f: (b,f)}
a path from a to e is [a, b, c, e]
